In [1]:
import os
import numpy as np
from PIL import Image
from cellpose import denoise

In [2]:
def load_images_from_folder(folder):
    """Load all images from a folder as NumPy arrays and store their filenames."""
    images = []
    filenames = []
    for filename in sorted(os.listdir(folder)):  # Sort to maintain order
        if filename.endswith(('.tif', '.png', '.jpg', '.jpeg')):
            img_path = os.path.join(folder, filename)
            img = Image.open(img_path).convert('RGB')  # Convert to RGB
            img_array = np.array(img)  # Convert image to NumPy array
            images.append(img_array)
            filenames.append(filename)  # Store original filename
    return images, filenames  # Return images as a list, not a single NumPy array

def process_cellpose_output(img_array):
    """Convert Cellpose output to a format compatible with PIL while preserving channels."""
    img_array = np.squeeze(img_array)  # Remove singleton dimensions

    # Normalize if float32 (scale to 0-255)
    if img_array.dtype == np.float32:
        img_array = (img_array - img_array.min()) / (img_array.max() - img_array.min()) * 255
        img_array = img_array.astype(np.uint8)  # Convert to 8-bit
    
    # If the image is multi-channel (e.g., RGB), ensure all channels are handled correctly
    if img_array.ndim == 3:  # Multi-channel (RGB or other)
        img_array = np.clip(img_array, 0, 255)  # Ensure values are within byte range (0-255)

    return img_array

def save_restored_images(filenames, restored_images, output_folder):
    """Save restored images as .tif files with original names."""
    os.makedirs(output_folder, exist_ok=True)  # Ensure output folder exists

    for filename, img_array in zip(filenames, restored_images):
        save_path = os.path.join(output_folder, filename)  # Keep original name
        img = Image.fromarray(img_array.astype(np.uint8))  # Convert to 8-bit TIFF
        img.save(save_path)

In [13]:
# === RUN PIPELINE ===
input_folder = r"D:\images\tnia-python-images\imagesc\2025_02_13_cellpose_restoration"
output_folder = r"D:\images\tnia-python-images\imagesc\2025_02_13_cellpose_restoration\restored"

# Step 1: Load Images
imgs, filenames = load_images_from_folder(input_folder)

# Step 2: Restore Images using Cellpose (Process Each Image Individually)
dn = denoise.DenoiseModel(model_type="denoise_cyto2", gpu=True)

restored_imgs = []
for img in imgs:
    img_dn = dn.eval(np.expand_dims(img, axis=0), channels=[1,2,3], diameter=15.)  # Process one image at a time
    #img_dn = dn.eval(np.expand_dims(img, axis=0), channels=[1,2,3], diameter=15.)  # Process one image at a time
    print(img_dn.shape)
    print(img_dn[0].shape)
    restored_img = process_cellpose_output(img_dn)  # Get the first (and only) image in the batch
    restored_imgs.append(restored_img)  # Store the restored image

# Step 3: Save Restored Images with Original Names
save_restored_images(filenames, restored_imgs, output_folder)


(1462, 1702, 2)
(1702, 2)
(1794, 1737, 2)
(1737, 2)
(1785, 1676, 2)
(1676, 2)
(1706, 1789, 2)
(1789, 2)
(1072, 1433, 2)
(1433, 2)
(1207, 1565, 2)
(1565, 2)
(1786, 1680, 2)
(1680, 2)
(1597, 1757, 2)
(1757, 2)
(1782, 1706, 2)
(1706, 2)
(1381, 1445, 2)
(1445, 2)
(1492, 1713, 2)
(1713, 2)


In [7]:
help(denoise.DenoiseModel)

Help on class DenoiseModel in module cellpose.denoise:

class DenoiseModel(builtins.object)
 |  DenoiseModel(gpu=False, pretrained_model=False, nchan=1, model_type=None, chan2=False, diam_mean=30.0, device=None)
 |  
 |  DenoiseModel class for denoising images using Cellpose denoising model.
 |  
 |  Args:
 |      gpu (bool, optional): Whether to use GPU for computation. Defaults to False.
 |      pretrained_model (bool or str or Path, optional): Pretrained model to use for denoising.
 |          Can be a string or path. Defaults to False.
 |      nchan (int, optional): Number of channels in the input images, all Cellpose 3 models were trained with nchan=1. Defaults to 1.
 |      model_type (str, optional): Type of pretrained model to use ("denoise_cyto3", "deblur_cyto3", "upsample_cyto3", ...). Defaults to None.
 |      chan2 (bool, optional): Whether to use a separate model for the second channel. Defaults to False.
 |      diam_mean (float, optional): Mean diameter of the objects in

In [4]:

save_restored_images(filenames, restored_imgs, output_folder)